# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv", names=["Title", "Year", "Synopsis", "Review"], skiprows=[0])
    df_eng['Original Language'] = 'en'
    
    
    df_fr = pd.read_csv("data/movie_reviews_fr.csv", names=["Title", "Year", "Synopsis", "Review"], skiprows=[0])
    df_fr['Original Language'] = 'fr'

    
    df_es = pd.read_csv("data/movie_reviews_sp.csv", names=["Title", "Year", "Synopsis", "Review"], skiprows=[0])
    df_es['Original Language'] = 'sp'
    
    
    
    df = pd.concat([df_eng, df_fr, df_es])
    df = df.reset_index(drop=True)
    return df

df = preprocess_data()

In [3]:
df.head()

,Title,Year,Synopsis,Review,Original Language
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df[df['Original Language'].str.contains('fr')]['Review']
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# filter synopsis in French and translate to English
fr_synopsis = df[df['Original Language'].str.contains('fr')]['Synopsis']
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df[df['Original Language'].str.contains('sp')]['Review']
es_reviews_en = es_reviews.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df[df['Original Language'].str.contains('sp')]['Synopsis']
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df.loc[df['Original Language'].str.contains('fr'), 'Review'] = fr_reviews_en
df.loc[df['Original Language'].str.contains('fr'), 'Synopsis'] = fr_synopsis_en

df.loc[df['Original Language'].str.contains('sp'), 'Review'] = es_reviews_en
df.loc[df['Original Language'].str.contains('sp'), 'Synopsis'] = es_synopsis_en


/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [6]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",fr
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",en
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",sp
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",sp
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [7]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "siebert/sentiment-roberta-large-english"
sentiment_classifier = pipeline(
    model=model_name
)


# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    return classifier(text)

In [8]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
sentiments_col = df['Review']
sentiments_col = sentiments_col.apply(lambda x: analyze_sentiment(x, sentiment_classifier))
sentiments_col = sentiments_col.apply(lambda x:  x[0]['label'])
df['Sentiment'] = sentiments_col

In [9]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en,NEGATIVE
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",sp,NEGATIVE
10,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",fr,POSITIVE
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp,NEGATIVE
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en,POSITIVE
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp,POSITIVE
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en,POSITIVE
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp,NEGATIVE
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",en,NEGATIVE
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",sp,POSITIVE


In [10]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)